# Holmberg II X-1

In [ ]:
import time
import glob
import importlib
import tempfile
import numpy as np
import xarray as xr
import astropy
import astropy.io.fits as fits
from astropy.table import Table
from astropy.wcs import WCS
import matplotlib.pyplot as plt

from astro import astroutils
from astro import mastrometry
from astro import mimage
from astro import mphot

from utils import mutils
from utils import mlogging 

from astro.astroutils import jd2UTCtimestamp, UTCtimestamp2jd

from constants import dr_tek2k, plate_scales

log = mlogging.getLogger('photometry')

%matplotlib ipympl

In [3]:
data = r'/Users/canis/dev/holmberg/data/test/calibrated/g21d018.050.c.fits'
hdu = fits.open(data)[1]
image = hdu.data

In [5]:
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std
bkg_sigma = mad_std(image)  
daofind = DAOStarFinder(fwhm=4., threshold=3. * bkg_sigma)  
sources = daofind(image)  
for col in sources.colnames:  
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)  

 id  xcentroid ycentroid sharpness  ... sky  peak    flux       mag    
---- --------- --------- ---------- ... --- ----- --------- -----------
   1 1.5704267 1.5704267 0.24378076 ...   0 32768 3888.5189  -8.9744605
   2 1.4295733 1.5743044 0.24378076 ...   0 32768 3888.5189  -8.9744605
   3  1142.003 1.7700926 0.32877528 ...   0    24 1.7158099 -0.58617292
   4 2108.5704 1.5704267 0.24378076 ...   0 32768 3888.5189  -8.9744605
   5 2109.4296 1.5704267 0.24378076 ...   0 32768 3888.5189  -8.9744605
   6 1199.9654 2.2054594 0.41186025 ...   0    97 7.5742017  -2.1983422
   7 1425.6299 6.1299456 0.63502731 ...   0    81 7.5090734  -2.1889659
   8 1905.9681 12.128973 0.69440072 ...   0   222 23.836698  -3.4431153
   9 1209.6368 14.714499 0.49173585 ...   0    41 1.9315983 -0.71479203
  10 1258.8487 14.724205 0.45644814 ...   0   197 17.999507  -3.1381515
 ...       ...       ...        ... ... ...   ...       ...         ...
1680 578.56856  2013.498 0.43200553 ...   0    40 2.2544703 -0.8

In [6]:
from photutils.aperture import aperture_photometry, CircularAperture
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))  
apertures = CircularAperture(positions, r=4.)  
phot_table = aperture_photometry(image, apertures)  
for col in phot_table.colnames:  
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)  

 id   xcenter   ycenter  aperture_sum
        pix       pix                
---- --------- --------- ------------
   1 1.5704267 1.5704267     773909.8
   2 1.4295733 1.5743044    774410.11
   3  1142.003 1.7700926    897.81503
   4 2108.5704 1.5704267    944097.38
   5 2109.4296 1.5704267    944097.38
   6 1199.9654 2.2054594    2129.1074
   7 1425.6299 6.1299456    1326.8494
   8 1905.9681 12.128973    1986.9173
   9 1209.6368 14.714499     1366.238
  10 1258.8487 14.724205    3720.2659
 ...       ...       ...          ...
1681 705.71013 2019.8938    1172.1404
1682 7.2903459  2020.861    23280.755
1683 930.57224 2030.9553    4132.4965
1684 1235.3881 2038.9551    6466.6133
1685 1.5704267 2045.4296    773829.97
1686 1.4295733 2045.4296    773419.41
1687 17.568153   2045.51     693.9257
1688 1553.1666 2045.3915    874.12664
1689 2108.5704 2045.4296    944617.46
1690 2109.4296 2045.4296    944097.38
Length = 1690 rows


In [ ]:
plt.imshow(image, cmap='gray_r', origin='lower')
apertures.plot(color='blue', lw=1.5, alpha=0.5)

In [2]:
disk_radius = 3.8
sky_annulus_inner = 7
sky_annulus_outer = 15


3.8 6.999999999999999 15.0
6.08 11.2 24.000000000000004
2.8085454545454547 5.173636363636363 11.086363636363636
2.0596 3.7939999999999996 8.13
